Tạo kết nối với Plaid

In [2]:
import os
from plaid.model import *
from plaid.configuration import Configuration
from plaid.api_client import ApiClient
from plaid.api import plaid_api


# Lấy thông tin từ biến môi trường
client_id = os.getenv("PLAID_CLIENT_ID")
secret = os.getenv("PLAID_SECRET")
environment = os.getenv("PLAID_ENV", "sandbox")  # Mặc định là sandbox

# Cấu hình Plaid API
configuration = Configuration(
    host=f"https://{environment}.plaid.com",
    api_key={
        "clientId": client_id,
        "secret": secret
    }
)
api_client = ApiClient(configuration)
plaid_client = plaid_api.PlaidApi(api_client)


Tạo Public Token và Exchange Access Token

In [9]:
from plaid.model.products import Products
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from plaid.model.country_code import CountryCode
from plaid.model.link_token_create_request import LinkTokenCreateRequest


def create_link_token():
    try:
        request = LinkTokenCreateRequest(
            client_name="MyFinanceApp",  # Tên ứng dụng của bạn
            language="en",
            country_codes=[CountryCode('US')],  # Mã quốc gia hợp lệ
            user=LinkTokenCreateRequestUser(client_user_id="12345"),  # ID người dùng hợp lệ
            products=[Products('transactions')]  # Các sản phẩm bạn muốn truy cập
        )

        response = plaid_client.link_token_create(request)
        print("Link Token Created:", response.link_token)
        return response.link_token
    except Exception as e:
        print("Error:", e)

# Gọi hàm
create_link_token()


Error: expected string or bytes-like object


In [4]:
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest


def exchange_public_token(public_token):
    request = ItemPublicTokenExchangeRequest(public_token=public_token)
    response = plaid_client.item_public_token_exchange(request)
    print("Access Token:", response.access_token)
    return response.access_token


Lấy danh sách tài khoản

In [5]:
from plaid.model.accounts_get_request import AccountsGetRequest


def get_accounts(access_token):
    request = AccountsGetRequest(access_token=access_token)
    response = plaid_client.accounts_get(request)
    for account in response.accounts:
        print(f"Account Name: {account.name}, Balance: {account.balances.current}")


Lấy dữ liệu giao dịch

In [6]:
from plaid.model.transactions_get_request import TransactionsGetRequest


def get_transactions(access_token, start_date, end_date):
    request = TransactionsGetRequest(
        access_token=access_token,
        start_date=start_date,
        end_date=end_date,
    )
    response = plaid_client.transactions_get(request)
    for transaction in response.transactions:
        print(f"Date: {transaction.date}, Name: {transaction.merchant_name}, Amount: {transaction.amount}")


 Chạy chương trình

In [7]:
if __name__ == "__main__":
    # 1. Tạo Link Token và hiển thị cho người dùng
    link_token = create_link_token()

    # 2. Người dùng hoàn tất và cung cấp Public Token
    public_token = input("Enter the Public Token after connecting your account: ")
    
    # 3. Exchange Public Token lấy Access Token
    access_token = exchange_public_token(public_token)

    # 4. Lấy danh sách tài khoản
    print("\nAccounts:")
    get_accounts(access_token)

    # 5. Lấy giao dịch
    print("\nTransactions:")
    get_transactions(access_token, "2023-01-01", "2023-12-31")


TypeError: expected string or bytes-like object